# 导入模块

In [104]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False
import json
import warnings
warnings.filterwarnings("ignore")
import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [105]:
clean = Data_Clean()

In [106]:
f_path="F:/myfile/p站数据/当月数据"
#f_path="F:/下班前订单数据"
#f_path="F:/订单日报/临时导出"

In [107]:
import os
excel_names = []
excel_date = []
for excel_name in os.listdir(f_path):
    # excel_names.append(excel_name)
    # excel_date.append(excel_name.split('.')[0])
    if not excel_name.startswith('~$') and excel_name.endswith('.xlsx'):
        excel_names.append(excel_name)
        excel_date.append(excel_name.split('.')[0])
print(excel_names)
print(excel_date)


['1201-1215.xlsx']
['1201-1215']


In [108]:
# df_list=[]
# for excel_name in excel_names:
#     # 读取每个excel到df
#     excel_path = f"{f_path}/{excel_name}"
#     print(excel_path)
#     df_split = pd.read_excel(excel_path,skiprows=0)
#     print(df_split)
#     df_list.append(df_split)
#     print(df_list)

# df = pd.concat(df_list,ignore_index=True)

In [109]:
df = pd.read_excel(f'F:/myfile/p站数据/当月数据/{excel_date[-1]}.xlsx')

In [110]:
df["下单日期"]=df["下单时间"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
df["月份"]=df["下单日期"].dt.month

In [111]:
df = df[df['商品属性'].notnull()]

# 信息获取

In [112]:
df["拒绝理由"]=df["备注信息"].str.split("拒绝理由：").str[1]
df["拒绝理由"]=df["拒绝理由"].str.replace("[","").str.replace("]","").str.replace('"','')
df["取消原因"]=df["备注信息"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["备注信息"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
#df["无法联系原因"]=df["备注信息"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["下单商品"].str.contains('99新')|df["下单商品"].str.contains('95新')|df["下单商品"].str.contains('准新'),"二手","全新")
df.loc[:,"租赁方案"]=np.where(df["商品属性"].str.contains('租完即送'),"租完即送","租完归还")
# 京东，多出一个随租随还方案
df.loc[:,"租赁方案"]=np.where(df["商品属性"].str.contains('随租随还'),"随租随还",df["租赁方案"])
df.loc[:,"押金类型"]=np.where(df["预授权占用自有资金"]>0,"部分免押","全免押")
df.loc[:,"是否使用优惠券"]=np.where((df["设置买断金额"]>df["总租金"])&(df["租赁方案"]=="租完即送"),"使用优惠券","未使用优惠券")
df.loc[:,"是否进行预授权"]=np.where(df["预授权占用自有资金"].isnull(),"未预授权","已预授权")

In [113]:
df.租赁方案.value_counts()

租赁方案
租完归还    105506
租完即送     13254
随租随还       150
Name: count, dtype: int64

In [114]:
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["商品属性"]),axis=1)

In [115]:
def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["商品属性"]),axis=1)

In [116]:
df.loc[:,"进件"]=np.where((df["订单状态"]=="待支付")|(df["订单状态"]=="订单取消"),"未进件","进件")

# 渠道归属

In [117]:
df.loc[:,"来源渠道"]=df["来源渠道"].fillna("未知渠道")

In [118]:
def qudao_type(a,b):
    a = str(a)
    b = str(b)
    if '京东活动' in b:
        return '京享租'
    elif "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "苹果BD" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"   
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "支付宝直播" in a or '支付宝直播' in b or '支付宝直播商品' in b:
        return "支付宝直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in b:
        return "生活号"
    elif "社群" in b or '支群' in b:
        return "支付宝社群"
    
    else :
        return a

In [119]:
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["活动名称"]),axis=1)

In [120]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["订单状态"].map(dict_status_code)

In [121]:
df.sort_values(by=["下单日期","状态编码"],inplace=True)

# 删除重复数据

In [122]:
#删除重复单号
df.drop_duplicates(subset=["单号"],inplace=True)
#删除身份证空值行
df.dropna(subset=["下单用户身份证号"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['备注信息'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['活动名称']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['活动名称']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['活动名称']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['活动名称']=="线下门店3个月试行"].index,inplace=True)
#删除订单状态空值行
df.dropna(subset=["订单状态"],axis=0,inplace=True)
df.drop(df[df['下单用户'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华","邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文","王家伟"])].index,inplace=True)

In [123]:
#删除重复订单
df.drop_duplicates(subset=["单号"],inplace=True)
df.drop_duplicates(subset=["下单用户","下单手机","下单用户身份证号","下单日期"],keep="last",inplace=True)

# 定义状态

In [124]:
def reject_type(a,b,c,d,e,f,g):
    if a is not np.nan and b=="未进件":
        return "前置拦截"
    elif g is np.nan and b=="未进件":
        return "进件前取消"
    elif a is not np.nan and b=="进件":
        return "机审强拒"
    elif c is not np.nan and"抖音非免押用户" in c:
        return "抖音非免押拒绝"
    elif a is np.nan and b=="进件" and c is not np.nan and '已退款' in e:
        return "人审拒绝"
    elif a is np.nan and b=="进件" and d is not np.nan:
        return "客户取消"
    #elif a is np.nan and b=="进件" and f is not np.nan:
        #return "无法联系"
    elif g is not np.nan and b=="进件" and "蚂蚁数控风险等级" in g:
        return "出库前风控强拒"
    elif g is not np.nan and b=="进件" and "命中出库前风控流强拒" in g:
        return "出库前风控强拒"
    elif a is np.nan and b=="进件" and "待审核" in e:
        return "待审核"
    elif a is np.nan and b=="进件" and "待发货" in e:
        return "出库"
    elif a is np.nan and b=="进件" and "待收货" in e:
        return "出库"
    elif a is np.nan and b=="进件" and "租赁中" in e:
        return "出库"
    elif a is np.nan and b=="进件" and "已完成" in e:
        return "出库"
    elif a is np.nan and b=="未进件":
        return "未进件"
    else :
        return e
    
    

In [125]:
df.loc[:,"审核状态"]=df.apply(lambda x: reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["订单状态"],x["订单状态"],x["备注信息"]),axis=1)

In [126]:

fw_path = 'F:/myfile/p站数据/服务订单'
fw_excel_names = []
fw_excel_date = []
for excel_name in os.listdir(fw_path):
    fw_excel_names.append(excel_name)
    fw_excel_date.append(excel_name.split('.')[0])
# print(excel_names)
print(fw_excel_date)


['~$服务订单0611-0617', '服务订单1201-1215']


In [127]:
f_path_insure=f"F:/myfile/p站数据/服务订单/{fw_excel_date[-1]}.xlsx"
df_insure=pd.read_excel(f_path_insure,skiprows=0)
df_insure1=pd.merge(df,df_insure,left_on="单号", right_on="租赁订单号",how="left")
df=df_insure1

In [128]:
import datetime

Today=datetime.date.today()
str(Today)
Now=str(datetime.datetime.now() )
Now

'2025-12-17 14:50:59.700315'

# 删除商家数据

In [129]:
df_contain = df.copy()
# df_contain[df_contain['商家']=='星晟数码'].shape
# 选取商家为人人享租，喜卓灵新租机的订单数据，宏生需求，2025-09-11
# df_ = df[df['商家'].isin(['人人享租','喜卓灵新租机'])]
# df_.商家.unique()

## 获取出库类台账数据（宏生）

In [130]:
# import pymysql
# def query(sql,
#         host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
#         user="ylc",
#         password="1O8t5lcJ5aMhwwPEUUjS",
#         database = '' ,
#         port=3306
#         ):   
#     conn = pymysql.connect(
#         host=host,
#         user=user,
#         port=port,
#         password=password,
#         max_allowed_packet=1073741824,
#         charset="utf8")
#     try:
#         df = pd.read_sql(sql, con=conn)
#         conn.close()
#     except:
#         print('error')
#         conn.close()
#         raise
#     return df

In [131]:
# # 订单号  租金om.all_money	已付租金tos.real_pay_money(tos.status=3)	期数tos.sort  碎屏险（是否有碎屏险）buy_service_product，1否2是

# sql_ck = '''
# select om.id order_id,om.order_number 订单号, om.all_money, tos.real_pay_money, tos.sort, om.buy_service_product,om.status

# from db_digua_business.t_order om
# left join db_digua_business.t_order_stages tos on om.id=tos.order_id
# where om.user_mobile is not null 
# and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-08-01'
# and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-09-10'
# and tos.status=3
# '''
# df_ck = query(sql_ck)

# df_ck.shape

## 去重

In [132]:
# # 按照order_id、sort去重
# duplicates = df_ck[df_ck.duplicated(subset=['order_id', 'sort'], keep=False)]
# if not duplicates.empty:
#     print("发现重复数据，数量：", len(duplicates))
#     # 可以选择保留最新的记录（根据实际业务逻辑调整）
#     # 这里假设reality_refund_date不为空的是更新的记录
#     df_ck = df_ck.sort_values(['order_id', 'sort', 'refund_date'], ascending=[True, True, False])
#     # 去重，保留每个(order_id, sort)的第一条记录（即最新的）
#     df_ck = df_ck.drop_duplicates(subset=['order_id', 'sort'], keep='first')
    
# df_ck["是否出库"] = np.where(df_ck["status"].isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)
# df_ck["是否有碎屏险"] = np.where(df_ck["buy_service_product"] == 2, "有碎屏险", "无碎屏险")
# df_ck.loc[:,"已付租金"]=df_ck["real_pay_money"].fillna(0)
# df_ck.loc[:,"期数"] = df_ck["sort"]
# df_ck.loc[:,"总租金"]=df_ck["all_money"]
# df_ck = df_ck[df_ck['是否出库'] == 1][['order_id', '订单号', '总租金', '已付租金', '期数', '是否有碎屏险']]
# df_ck.columns

In [133]:

# dfck=pd.merge(df_ck,df_,left_on="订单号", right_on="单号")
# dfck.shape


In [134]:

df.drop(df[df['商家']=="深圳优优大数据科技有限公司"].index,inplace=True)
df.drop(df[df['商家']=="优优2店"].index,inplace=True)
df.drop(df[df['商家']=="探路者自营账户"].index,inplace=True)
df.drop(df[df['商家']=="苏州蚁诺宝"].index,inplace=True)
df.drop(df[df['商家']=="租着用电脑数码"].index,inplace=True)
df.drop(df[df['商家']=="北京海鸟窝科技有限公司"].index,inplace=True)
df.drop(df[df['商家']=="澄心优租"].index,inplace=True)
df.drop(df[df['商品型号'].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
# 趣智数码  单
df.drop(df[df['商家']=="趣智数码"].index,inplace=True)
df.drop(df[df['商家']=="格木木二奢名品"].index,inplace=True)
df.drop(df[df['商家']=="广州康基贸易有限公司"].index,inplace=True)
df.drop(df[df['商家']=="汇客好租"].index,inplace=True)
df.drop(df[df['商家']=="线下小店"].index,inplace=True)
df.drop(df[df['商家']=="乙辉数码"].index,inplace=True)
df.drop(df[df['商家']=="兴鑫兴通讯"].index,inplace=True)
df.drop(df[df['商家']=="南京聚格网络科技"].index,inplace=True)
df.drop(df[df['商家']=="呱子笔记本电脑"].index,inplace=True)

df.drop(df[df['商家'] == "星晟数码"].index, inplace=True)
df.drop(df[df['商家'] == "蘑菇时间"].index, inplace=True)
# df = clean.drop_merchant(df)

In [135]:
df_j=df[df["进件"]=="进件"]

In [136]:
# 商家拒量
df.drop(df[df['商家']=="小蚂蚁租机"].index,inplace=True)
df.drop(df[df['商家']=="兴鑫兴通讯"].index,inplace=True)
df.drop(df[df['商家']=="人人享租"].index,inplace=True)
df.drop(df[df['商家']=="崇胜数码"].index,inplace=True)
df.drop(df[df['商家']=="喜卓灵租机"].index,inplace=True)
df.drop(df[df['商家']=="喜卓灵新租机"].index,inplace=True)

In [137]:
df_j[df_j.归属渠道=='搜索渠道'].shape

(6833, 88)

# 以下为出库相关数据，要统计到出库相关的数据，必须运行到以下读取出库台账的代码

In [138]:
#读取导入出库台账数据
# f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
# df_ck = pd.read_excel(f_path_ck,sheet_name="2023",skiprows=0) 
f_path_ck="F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
df_ck=pd.read_excel(f_path_ck,sheet_name="2025",skiprows=0)
dfck=pd.merge(df_ck,df,left_on="订单号", right_on="单号")
dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.shape


(2999, 106)

In [139]:
dfck.订单状态.value_counts()

订单状态
租赁中    2316
待收货     345
已退款     227
已完成      67
待发货      39
检测中       5
Name: count, dtype: int64

# 检查出库订单中有没有客户取消的单，即订单状态为”已退款"的单，找出来删除后重新运行上一块导入出库台账的代码

In [140]:
dfck.drop(dfck[dfck["订单状态"]=="已退款"].index,inplace=True)

# 剔除拒收!!!!
dfck.drop(dfck[dfck["订单状态"]=="已完成"].index,inplace=True)

In [141]:
dfck.订单状态.value_counts()

订单状态
租赁中    2316
待收货     345
待发货      39
检测中       5
Name: count, dtype: int64

In [142]:
try:
    dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)
except:
    dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)

# 苹果15系列商品型号-进件

In [143]:
# #苹果15系列机型进件
# df_j["型号内存"]=df_j["商品型号"]+"_"+df_j["内存"]
# df_jap15=df_j[df_j["型号内存"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(df_jap15["下单日期"],df_jap15["型号内存"],margins=True).to_excel(f"F:苹果15进件.xlsx")

In [144]:
# #华为进件
# df_jhw=df_j[df_j["商品型号"].str.contains(pat="Mate60",regex=False)==True]
# pd.crosstab(df_jhw["商品型号"],df_jhw["内存"],margins=True)


In [145]:
# df_jhw.groupby(["商品型号","内存","颜色"]).agg({"单号":np.size}).unstack(-1)

# 苹果15系列商品型号-出库

In [146]:
# # 苹果15系列商品型号-出库
# df_ap15=dfck[dfck["商品型号"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(df_ap15["下单日期"],df_ap15["商品型号"],margins=True)

In [147]:
# #华为出库
# dfckhw=dfck[dfck["商品型号"].str.contains(pat="Mate60",regex=False)==True]

# pd.crosstab(dfckhw["商品型号"],dfckhw["内存_x"],margins=True)

In [148]:
# dfckhw.groupby(["商品型号","内存_y","颜色_x"]).agg({"订单号":np.size}).unstack(-1)

# 某渠道苹果15系列商品型号-进件

In [149]:
#苹果15系列商品型号-去重订单
# df_qd=df_j[df_j["来源渠道"]=="芝麻租物"]
# df_jap15=df_qd[df_qd["商品型号"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(df_jap15["下单日期"],df_jap15["商品型号"],margins=True).to_excel("F:/芝麻租物/2024.3芝麻渠道15进件.xlsx")

# 某渠道苹果15系列商品型号-去重订单


In [150]:
# # 某渠道苹果15系列商品型号-进件
# df_qd=df[df["来源渠道"]=="芝麻租物"]
# df_ap15=df_qd[df_qd["商品型号"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(df_ap15["下单日期"],df_ap15["商品型号"],margins=True).to_excel("F:/线上/芝麻租物/2024.3芝麻渠道15去重订单.xlsx")

# 某渠道苹果15系列商品型号-出库

In [151]:
# # 某渠道苹果15系列商品型号-出库
# dfck_qd=dfck[dfck["来源渠道"]=="芝麻租物"]
# dfck_jap15=dfck_qd[dfck_qd["商品型号"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(dfck_jap15["下单日期"],dfck_jap15["商品型号"],margins=True).to_excel("F:/线上/芝麻租物/2024.3芝麻渠道15出库.xlsx")

# 宏生碎屏险数据，导出来直接甩给他就可以了

In [152]:
# #总体出库订单碎屏险购买数据
# df_bx_ck=pd.crosstab(dfck["下单日期"],dfck["是否购买服务商品：1、否；2、是"],margins=True)
# df_bx_ck=df_bx_ck.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
# df_s3=pd.crosstab(dfck["下单日期"],dfck["服务订单状态"],margins=True)
# df_s3=df_s3.drop(columns={"All"})
# df_s2merge=pd.merge(df_bx_ck,df_s3,left_index=True, right_index=True)
# df_s2merge["最终实际支付碎屏险出库"]=df_s2merge["待确认"]+df_s2merge["服务中"] #+df_s2merge["待支付"]
# df_s2merge["实际支付碎屏险比例"]=df_s2merge["最终实际支付碎屏险出库"]/df_s2merge["合计出库"]
# df_s2merge["实际支付碎屏险比例"]=df_s2merge["实际支付碎屏险比例"].apply(lambda x:format(x,".2%"))
# df_s2merge

In [153]:
# #芝麻租物出库订单碎屏险购买数据
# dfckzw=dfck[dfck["归属渠道"]=="芝麻租物"]
# df_zw3=pd.crosstab(dfckzw["下单日期"],dfckzw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})
# df_zws3=pd.crosstab(dfckzw["下单日期"],dfckzw["服务订单状态"],margins=True)
# df_zws3=df_zws3.drop(columns={"All"})
# df_zws2merge=pd.merge(df_zw3,df_zws3,left_index=True, right_index=True)
# df_zws2merge["最终实际支付碎屏险出库"]=df_zws2merge["服务中"]+df_zws2merge["待确认"]#+df_s2merge["待支付"]
# df_zws2merge

In [154]:
# #Mate60Pro 购买碎屏险数据
# dfckhw=dfck[dfck["商品型号"].str.contains(pat="Mate60Pro",regex=False)==True]
# df_zw3=pd.crosstab(dfckhw["下单日期"],dfckhw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})
# df_zws3=pd.crosstab(dfckhw["下单日期"],dfckhw["服务订单状态"],margins=True)
# df_zws3=df_zws3.drop(columns={"All"})
# df_zws2merge=pd.merge(df_zw3,df_zws3,left_index=True, right_index=True)
# df_zws2merge["最终实际支付碎屏险出库"]=df_zws2merge["服务中"]#+df_zws2merge["待确认"]
# df_zws2merge

In [155]:
# #Mate60 购买碎屏险数据
# dfckhw=dfck[dfck["商品型号"].str.contains(pat="Mate60",regex=False)==True]
# df_zw3=pd.crosstab(dfckhw["下单日期"],dfckhw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})
# df_zws3=pd.crosstab(dfckhw["下单日期"],dfckhw["服务订单状态"],margins=True)
# df_zws3=df_zws3.drop(columns={"All"})
# df_zws2merge=pd.merge(df_zw3,df_zws3,left_index=True, right_index=True)
# # df_zws2merge["最终实际支付碎屏险出库"]=df_zws2merge["服务中"]   # df_zws2merge["待确认"]+
# df_zws2merge

# 总体碎屏险数据

In [156]:
# #总体碎屏险数据,宏生需求
# df_bx=pd.crosstab(df["下单日期"],df["是否购买服务商品：1、否；2、是"],margins=True)
# df_bx=df_bx.rename(columns={1:"不购买碎屏险订单",2:"购买碎屏险订单","All":"合计下单",})
# df_bx_j=pd.crosstab(df_j["下单日期"],df_j["是否购买服务商品：1、否；2、是"],margins=True)
# df_bx_j=df_bx_j.rename(columns={1:"不购买碎屏险进件",2:"购买碎屏险进件","All":"合计进件",})
# df_bx_ck=pd.crosstab(dfck["下单日期"],dfck["是否购买服务商品：1、否；2、是"],margins=True)
# df_bx_ck=df_bx_ck.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
# df_spx1=pd.merge(df_bx,df_bx_j,left_index=True, right_index=True)
# df_spx2=pd.merge(df_spx1,df_bx_ck,left_index=True, right_index=True)#.to_excel(f"F:/碎屏险/总体碎屏险数据{excel_date[-1]}.xlsx")

In [157]:
# df_s1=pd.crosstab(df["下单日期"],df["服务订单状态"],margins=True)
# df_s1=df_s1.rename(columns={"All":"合计购买碎屏险订单"})
# df_s2=pd.crosstab(df_j["下单日期"],df_j["服务订单状态"],margins=True)
# df_s2=df_s2.rename(columns={"All":"合计购买碎屏险进件"})
# df_s3=pd.crosstab(dfck["下单日期"],dfck["服务订单状态"],margins=True)
# df_s3=df_s3.rename(columns={"All":"合计购买碎屏险出库"})
# df_s1merge=pd.merge(df_s1,df_s2,left_index=True, right_index=True)
# df_s2merge=pd.merge(df_s1merge,df_s3,left_index=True, right_index=True)
# df_s2merge["最终实际支付碎屏险出库"]=df_s2merge["待确认"]+df_s2merge["服务中"]
# df_s2merge

# 芝麻租物碎屏险数据

In [158]:
# #芝麻租物碎屏险数据,宏生需求
# dfzw=df[df["归属渠道"]=="芝麻租物"]
# df_zw1=pd.crosstab(dfzw["下单日期"],dfzw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw1=df_zw1.rename(columns={1:"不购买碎屏险订单",2:"购买碎屏险订单","All":"合计下单"})

# df_j_zw=df_j[df_j["归属渠道"]=="芝麻租物"]
# df_zw2=pd.crosstab(df_j_zw["下单日期"],df_j_zw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw2=df_zw2.rename(columns={1:"不购买碎屏险进件",2:"购买碎屏险进件","All":"合计进件"})
# df2=dfck[dfck["归属渠道"]=="芝麻租物"]
# df_zw3=pd.crosstab(df2["下单日期"],df2["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})
# df_zwspx1=pd.merge(df_zw1,df_zw2,left_index=True, right_index=True)
# df_zwspx2=pd.merge(df_zwspx1,df_zw3,left_index=True, right_index=True)#.to_excel(f"F:/碎屏险/芝麻租物碎屏险数据{excel_date[-1]}.xlsx")

In [159]:
# dfzw=df[df["归属渠道"]=="芝麻租物"]
# df_zws1=pd.crosstab(dfzw["下单日期"],dfzw["服务订单状态"],margins=True)
# df_zws1=df_zws1.rename(columns={"All":"合计购买碎屏险订单"})
# df_j_zw=df_j[df_j["归属渠道"]=="芝麻租物"]
# df_zws2=pd.crosstab(df_j_zw["下单日期"],df_j_zw["服务订单状态"],margins=True)
# df_zws2=df_zws2.rename(columns={"All":"合计购买碎屏险进件"})
# dfckzw=dfck[dfck["归属渠道"]=="芝麻租物"]
# df_zws3=pd.crosstab(dfckzw["下单日期"],dfckzw["服务订单状态"],margins=True)
# df_zws3=df_zws3.rename(columns={"All":"合计购买碎屏险出库"})
# df_zws1merge=pd.merge(df_zws1,df_zws2,left_index=True, right_index=True)
# df_zws2merge=pd.merge(df_zws1merge,df_zws3,left_index=True, right_index=True)
# df_zws2merge["最终实际支付碎屏险出库"]=+df_zws2merge["服务中"]+df_zws2merge["待确认"]
# df_zws2merge

In [160]:
#dfck[dfck["是否购买服务商品：1、否；2、是"]==2].to_excel(f"C:/Users/Administrator/Desktop/总体碎屏险数据出库_1119.xlsx")


# 月中\月底运营数据复盘 

# 月中/月底各渠道出库单数、租金均值、签约价均值、押金均值、租售比

In [161]:
#月中/月底各渠道出库单数、租金均值、签约价均值、押金均值、租售比
dfck_all=dfck.groupby(["月份","来源渠道"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"总押金":np.mean,}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["总押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,来源渠道,出库,总租金均值,签约价均值,总押金均值,租售比
0,12,八派信息,7,14585.077143,14585.077143,10570.428571,137.98%
1,12,其他渠道场景渠道。,7,7896.515714,13980.862857,10184.714286,77.53%
2,12,单人聊天会话中的小程序消息卡片（分享）,1,7500.550000,13796.600000,9999.000000,75.01%
3,12,小程序商家消息（服务提醒）,3,10519.106667,14716.473333,12184.740000,86.33%
4,12,我的小程序入口,5,9447.814000,14348.528000,11310.444000,83.53%
5,12,搜索-租手机,32,8596.950000,14381.512500,10499.000000,81.88%
6,12,搜索-租苹果手机,25,7880.039200,14040.564000,10199.000000,77.26%
7,12,搜索下挂-商品,262,7832.780267,14281.113893,10358.841298,75.61%
8,12,搜索直达banner,1,9000.630000,16556.200000,11999.000000,75.01%
9,12,支付宝客户端首页,23,9757.345652,14020.050435,10513.593043,92.81%


In [162]:
#月中/月底各渠道出库单数、租金均值、签约价均值、押金均值、租售比
dfck_all=dfck.groupby(["月份","归属渠道"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"总押金":np.mean,}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["总押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,归属渠道,出库,总租金均值,签约价均值,总押金均值,租售比
0,12,京享租,600,14456.930617,14456.930617,10620.428700,136.12%
1,12,八派信息,7,14585.077143,14585.077143,10570.428571,137.98%
2,12,其他渠道场景渠道。,7,7896.515714,13980.862857,10184.714286,77.53%
3,12,单人聊天会话中的小程序消息卡片（分享）,1,7500.550000,13796.600000,9999.000000,75.01%
4,12,小程序商家消息（服务提醒）,3,10519.106667,14716.473333,12184.740000,86.33%
5,12,我的小程序入口,5,9447.814000,14348.528000,11310.444000,83.53%
6,12,抖音渠道,2,6750.000000,11799.500000,9439.500000,71.51%
7,12,搜索渠道,799,8016.540088,14145.431164,10281.823579,77.97%
8,12,支付宝客户端首页,23,9757.345652,14020.050435,10513.593043,92.81%
9,12,支付宝直播,1,8197.200000,13796.620000,9999.000000,81.98%


# 月中/月底总体出库单数、租金均值、签约价均值、押金均值、租售比

In [163]:
dfck_all=dfck.groupby(["月份"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,12,2705,8813.851982,13758.248166,10240.17597,10250.003453,86.07%


In [164]:
#月中/月底总体出库单数、租金均值、签约价均值、押金均值、租售比
dfck_fjd = dfck[dfck.归属渠道!='京享租']
dfck_all=dfck_fjd.groupby(["月份"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,12,2105,7205.373511,13559.098774,10133.955344,10144.419059,71.10%


# 京享租

In [165]:
dfck_jd = dfck[dfck.归属渠道=='京享租']
dfck_all=dfck_jd.groupby(["月份"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,12,600,14456.930617,14456.930617,10612.833333,10620.4287,136.22%


# 月中/月底各租赁方案出库单数、租金均值、签约价均值、押金均值、租售比

In [166]:
#月中/月底各租赁方案出库单数、租金均值、签约价均值、押金均值、租售比
dfck_all=dfck_fjd.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,租赁方案,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,12,租完即送,61,13903.921967,13903.921967,10376.049180,10737.133115,134.00%
1,12,租完归还,2044,7005.465753,13548.808063,10126.730431,10126.730431,69.18%


# 京享租

In [167]:
dfck_all=dfck_jd.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,租赁方案,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,12,租完即送,599,14453.42596,14453.42596,10610.519199,10610.519199,136.22%
1,12,随租随还,1,16556.22000,16556.22000,11999.000000,16556.220000,137.98%


# 月中月底各渠道各租赁方案出库数

In [168]:
#月中月底各渠道租赁方案出库数
pd.crosstab(dfck["来源渠道"],dfck["租赁方案"],margins=True)

租赁方案,租完即送,租完归还,随租随还,All
来源渠道,,,,
八派信息,7,0,0,7
其他渠道场景渠道。,0,7,0,7
单人聊天会话中的小程序消息卡片（分享）,0,1,0,1
小程序商家消息（服务提醒）,1,2,0,3
我的小程序入口,1,4,0,5
搜索-租手机,4,28,0,32
搜索-租苹果手机,1,24,0,25
搜索下挂-商品,3,259,0,262
搜索直达banner,0,1,0,1


In [169]:
#月中月底各渠道租赁方案出库数
pd.crosstab(dfck["归属渠道"],dfck["租赁方案"],margins=True)

租赁方案,租完即送,租完归还,随租随还,All
归属渠道,,,,
京享租,599,0,1,600
八派信息,7,0,0,7
其他渠道场景渠道。,0,7,0,7
单人聊天会话中的小程序消息卡片（分享）,0,1,0,1
小程序商家消息（服务提醒）,1,2,0,3
我的小程序入口,1,4,0,5
抖音渠道,0,2,0,2
搜索渠道,41,758,0,799
支付宝客户端首页,7,16,0,23


# 月中月底各渠道租赁方案进件数

In [170]:
#月中月底各渠道租赁方案进件数
pd.crosstab(df_j["来源渠道"],df_j["租赁方案"],margins=True)

租赁方案,租完即送,租完归还,随租随还,All
来源渠道,,,,
京享租右卡,0,1,0,1
元火租超,1,0,0,1
八派信息,381,50,0,431
其他渠道场景渠道。,25,114,0,139
单人聊天会话中的小程序消息卡片（分享）,101,760,0,861
宜品花,9,6,0,15
小程序商家消息（服务提醒）,4,15,0,19
我的小程序入口,9,55,0,64
扫描二维码,20,2,0,22


In [171]:
df_zulin = pd.crosstab(df_j["来源渠道"],df_j["租赁方案"],margins=True)
# df_zulin.to_excel(f'F:/各渠道租赁进件数/{excel_date[-1]}进件数.xlsx')

In [172]:
#月中月底各渠道租赁方案进件数
pd.crosstab(df_j["归属渠道"],df_j["租赁方案"],margins=True)

租赁方案,租完即送,租完归还,随租随还,All
归属渠道,,,,
京享租,1137,7826,51,9014
京享租右卡,0,1,0,1
元火租超,1,0,0,1
八派信息,381,50,0,431
其他渠道场景渠道。,24,103,0,127
单人聊天会话中的小程序消息卡片（分享）,101,758,0,859
宜品花,9,6,0,15
小程序商家消息（服务提醒）,4,15,0,19
我的小程序入口,9,54,0,63


# 以下代码为测算毛利时候使用，使用前先从M站系统导出出库单获取成本数据，然后按顺序运行以下代码

In [173]:

ck_path = 'F:/myfile/p站数据/出库单'
ck_excel_names = []
ck_excel_date = []
for excel_name in os.listdir(ck_path):
    if not excel_name.startswith('~$') and excel_name.endswith('.xlsx'):
        ck_excel_names.append(excel_name)
        ck_excel_date.append(excel_name.split('.')[0])
# print(excel_names)
print(ck_excel_date)
os.listdir(ck_path)

['12月出库']


['12月出库.xlsx', '~$7月出库.xlsx']

In [201]:
f_path_cost=f"F:/myfile/p站数据/出库单/{ck_excel_date[-1]}.xlsx"
df_cost=pd.read_excel(f_path_cost,sheet_name='出库表',skiprows=0)
#df_cost.loc[:,"成本"]=np.where(df_cost["供应商"].str.contains('侯立')|df_cost["供应商"].str.contains('张陵')|df_cost["供应商"].str.contains('拾贝壳')|df_cost["供应商"].str.contains('大龙发'),(df_cost["设备成本"]-10)/1.025,df_cost["设备成本"])
# df_cost["成本"]=df_cost["设备成本"]
# df_cost["成本"]=df_cost["成本"].round()
dfck_cost=pd.merge(dfck,df_cost,left_on="订单号", right_on="订单号")
# 更改原因：由于台账存在成本字段与出库单表的成本字段发生冲突
dfck_cost.loc[:, "成本"]=np.where(dfck_cost["设备成本"] is np.nan, dfck_cost["成本"],dfck_cost["设备成本"])
dfck_cost["成本"]=dfck_cost["成本"].round()
dfck_cost.drop_duplicates(subset=["订单号"],inplace=True)
# 宏生需求
# dfck_cost.to_excel(f"F:/毛利/明细_{Today}.xlsx")

In [175]:
# 毛利计算过程，这块代码不能忽略
dfck_cost.loc[:,"原始毛利"]=dfck_cost["买断金额"]-dfck_cost["成本"]+dfck_cost["商品溢价费"]
# dfck_cost.loc[:,"残值机型"]=np.where(dfck_cost["商品型号"].str.contains('iP'),"苹果","非苹果")
dfck_cost["商品型号_all"] = dfck_cost["商品型号"][dfck_cost["商品型号"].str.contains('iP')==True] 
# dfck_cost.loc[:,"残值机型"]=np.where(dfck_cost["商品型号"].str.contains('iP'),  
#                                 np.where(dfck_cost["商品型号_all"].str.contains('16'), "苹果16", "苹果15"),"非苹果")
dfck_cost.loc[:,"残值机型"]=np.where(dfck_cost["商品型号"].str.contains('iP'),
                                np.where(dfck_cost["商品型号_all"].str.contains(r'iPhone 17|iPhone Air|iPhone17'), "苹果17",
                                    np.where(dfck_cost["商品型号_all"].str.contains('16'), "苹果16",
                                            np.where(dfck_cost['商品型号_all'].str.contains('15'), "苹果15",
                                                    np.where(dfck_cost['商品型号_all'].str.contains('14'), "苹果14", '苹果14以下')))),"非苹果")
'''
2025-8及以前的残值
def getrecanzhi(s):
    if s["租赁方案"]=="租完即送":
        return  0
    elif s["残值机型"]=="苹果16":
        return s["成本"]*0.6
    elif s["残值机型"]=="苹果15":
        return s["成本"]*0.47
    elif s["残值机型"]=="苹果14":
        return s["成本"]*0.4
    elif s["残值机型"]=="苹果14以下":
        return s["成本"]*0.375
    else:
        return s["成本"]*0.425
'''


def getrecanzhi(s):
    if s["租赁方案"]=="租完即送":
        return  0
    elif s["残值机型"]=="苹果17":
        return s["总押金"]*0.60
    elif s["残值机型"]=="苹果16":
        return s["成本"]*0.45
    elif s["残值机型"]=="苹果15":
        return s["成本"]*0.47
    elif s["残值机型"]=="苹果14":
        return s["成本"]*0.4
    elif s["残值机型"]=="苹果14以下":
        return s["成本"]*0.375
    else:
        return s["成本"]*0.425
dfck_cost.loc[:,"残值"]=dfck_cost.apply(getrecanzhi,axis=1)
dfck_cost.loc[:,"总应收"]=dfck_cost["总租金"]+dfck_cost["残值"]+dfck_cost["商品溢价费"]
dfck_cost.loc[:,"残值估算毛利"]=dfck_cost["总应收"]-dfck_cost["成本"]
def getremark(s):
    if s["成本"]==0:
        return s["已付金额"]
    else:
        return s["残值估算毛利"]
dfck_cost["残值估算毛利"]=dfck_cost.apply(getremark,axis=1)
dfck2 = dfck_cost[dfck_cost["成本"].notnull()]

In [176]:
# dfck_cost.groupby('月份').agg({'订单号': 'count', '总租金': 'sum', '残值': 'sum', '商品溢价费': 'sum', '总应收': 'sum', '成本': 'sum', '残值估算毛利': 'mean', '买断金额': 'sum', '原始毛利': 'mean'})

# 月中、月底总体数据测算毛利

In [177]:
# 月中、月底总体数据测算毛利
# 总应收=总租金+残值+溢价费
# 残值估算毛利=总应收-成本
# 买断估算毛利(原始毛利)=买断金额-成本+溢价费
# 租售比=总租金均值/商品押金均值
dfck2_fjd = dfck2[dfck2.归属渠道!='京享租']
dfck_zsb=dfck2_fjd.groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,订单号,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,12,1913,9651.466383,13567.896937,3916.430554,2546.608847,26.39%,40.58%


In [178]:
dfck2_jd = dfck2[dfck2.归属渠道=='京享租']
dfck_zsb=dfck2_jd.groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,订单号,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,12,550,10418.76,14422.993618,4004.233618,4022.294949,38.61%,38.43%


# 租物和非租物

In [179]:
# # 月中、月底总体数据测算毛利
# dfck2_zw = dfck2[dfck2.归属渠道=='芝麻租物']
# dfck_zsb=dfck2_zw.groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
# dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
# dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
# dfck_zsb.rename(columns={'月份':'月份','单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
# dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
# dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
# dfck_zsb


In [180]:
# dfck2_fzw = dfck2[dfck2.归属渠道!='芝麻租物']
# dfck_zsb=dfck2_fzw.groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
# dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
# dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
# dfck_zsb.rename(columns={'月份':'月份','单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
# dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
# dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
# dfck_zsb

# 月中、月底分租赁方案测算毛利

In [202]:
dfck_zsb=dfck2.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,租赁方案,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,12,租完即送,605,10380.385124,14393.197835,4012.812711,4012.812711,38.66%,38.66%
1,12,租完归还,1857,9639.946252,13550.669047,3910.722795,2499.592518,25.93%,40.57%
2,12,随租随还,1,12060.0,16556.220000,4496.22,14429.952000,119.65%,37.28%


In [182]:
#月中、月底分租赁方案测算毛利
dfck_zsb=dfck2_fjd.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,租赁方案,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,12,租完即送,56,10033.482143,14139.186071,4105.703929,4105.703929,40.92%,40.92%
1,12,租完归还,1857,9639.946252,13550.669047,3910.722795,2499.592518,25.93%,40.57%


# 京享租

In [183]:
dfck_zsb=dfck2_jd.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb
# 11月11后大部分订单租送

,月份,租赁方案,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,12,租完即送,549,10415.770492,14419.10796,4003.337468,4003.337468,38.44%,38.44%
1,12,随租随还,1,12060.0,16556.22000,4496.22,14429.952000,119.65%,37.28%


# 月中、月底分渠道测算毛利

In [184]:
# 月中、月底分渠道测算毛利
dfck_zsb=dfck2.groupby(["月份","来源渠道"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,来源渠道,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,12,八派信息,7,10560.0,14585.077143,4025.077143,4025.077143,38.12%,38.12%
1,12,其他渠道场景渠道。,7,10090.0,13980.862857,3890.862857,3722.894286,36.90%,38.56%
2,12,单人聊天会话中的小程序消息卡片（分享）,1,9940.0,13796.600000,3856.6,3559.950000,35.81%,38.80%
3,12,小程序商家消息（服务提醒）,3,10633.333333,14716.473333,4083.14,3885.373333,36.54%,38.40%
4,12,我的小程序入口,4,10487.5,14486.510000,3999.01,3946.680000,37.63%,38.13%
5,12,搜索-租手机,30,10337.666667,14420.506667,4082.84,3811.856667,36.87%,39.49%
6,12,搜索-租苹果手机,21,10075.714286,13955.623810,3879.909524,3604.604762,35.78%,38.51%
7,12,搜索下挂-商品,232,10366.577586,14319.976810,3953.399224,3655.218707,35.26%,38.14%
8,12,搜索直达banner,1,12102.0,16556.200000,4454.2,4098.030000,33.86%,36.81%
9,12,支付宝客户端首页,20,10045.55,13984.577000,3939.027,3939.061000,39.21%,39.21%


In [185]:
dfck_zsb=dfck2.groupby(["月份","归属渠道"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,归属渠道,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,12,京享租,550,10418.76,14422.993618,4004.233618,4022.294949,38.61%,38.43%
1,12,八派信息,7,10560.0,14585.077143,4025.077143,4025.077143,38.12%,38.12%
2,12,其他渠道场景渠道。,7,10090.0,13980.862857,3890.862857,3722.894286,36.90%,38.56%
3,12,单人聊天会话中的小程序消息卡片（分享）,1,9940.0,13796.600000,3856.6,3559.950000,35.81%,38.80%
4,12,小程序商家消息（服务提醒）,3,10633.333333,14716.473333,4083.14,3885.373333,36.54%,38.40%
5,12,我的小程序入口,4,10487.5,14486.510000,3999.01,3946.680000,37.63%,38.13%
6,12,抖音渠道,2,8221.095,11799.500000,3578.405,4192.605000,51.00%,43.53%
7,12,搜索渠道,724,10220.505525,14150.273177,3929.767652,3643.606547,35.65%,38.45%
8,12,支付宝客户端首页,20,10045.55,13984.577000,3939.027,3939.061000,39.21%,39.21%
9,12,未知渠道,8,11535.0,15866.302500,4331.3025,4091.773750,35.47%,37.55%


In [186]:
dfck_zsb =dfck2[dfck2["归属渠道"]=="搜索渠道"].groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,12,724,10220.505525,14150.273177,3929.767652,3643.606547,35.65%,38.45%


In [206]:
dfck2.下单日期.unique()

<DatetimeArray>
['2025-12-01 00:00:00', '2025-12-02 00:00:00', '2025-12-03 00:00:00',
 '2025-12-04 00:00:00', '2025-12-05 00:00:00', '2025-12-06 00:00:00',
 '2025-12-07 00:00:00', '2025-12-08 00:00:00', '2025-12-09 00:00:00',
 '2025-12-10 00:00:00', '2025-12-11 00:00:00', '2025-12-12 00:00:00',
 '2025-12-13 00:00:00', '2025-12-14 00:00:00']
Length: 14, dtype: datetime64[ns]

# 商家数据(每次跑都需要更改时间)

In [187]:

sql_merchant = """
select om.id as order_id, om.order_type
, replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.reject_check') end,'"','') as reject_check # 为1时表二级量，2025-12-04
from db_digua_business.t_order om
left join db_credit.risk risk on risk.order_id=om.id
where date_format(om.create_time,'%Y-%m-%d') >= '2025-12-01'
and date_format(om.create_time,'%Y-%m-%d') <= '2025-12-16'
and om.merchant_name in ('星晟数码', '汇客好租','云启德曜')
"""
df_merchant = clean.query(sql_merchant)
df_merchant.shape

(14751, 3)

In [188]:
df_contain.columns
# df_merchant

Index(['订单id', '单号', '外部订单号', '商家', '下单用户', '下单手机', '下单用户身份证号', '用户认证情况',
       '开始时间', '结束时间', '订单类型', '租期', '订单逾期天数', '总租金', '总押金', '商品押金', '总保险',
       '总运费', '总优惠', '订单状态', '是否买断', '下单时间', '收货人', '收货电话', '预授权支付单号', '活动名称',
       '收货省名称_x', '收货市名称_x', '收货区县名称', '收货人详细地址_x', '收货地址', '发货快递', '快递单号',
       '发货时间', '下单备注', '商品型号', '商品id', '下单商品', '商品属性', '商品数量', '租金单价', '买断金额',
       '预授权占用自有资金', '来源渠道', '备注信息', '用户实际支付金额', '联系人', '是否购买服务商品：1、否；2、是',
       '租物等级', '决策结果', '订单完成时间', '商品溢价费', '设置买断金额', '审核状态', '是否为资方订单',
       '第一期实付金额', '第二期应付金额', '第三期应付金额', '下单日期', '月份', '拒绝理由', '取消原因', '电审拒绝原因',
       '商品类型', '租赁方案', '押金类型', '是否使用优惠券', '是否进行预授权', '内存', '颜色', '进件', '归属渠道',
       '状态编码', '服务订单号', '租赁订单号', '服务商品名称', '服务商品价格', '服务商品改后价格', '服务订单状态',
       '服务时长（0表示长期）', '服务开始日期', '服务结束日期', '收货人姓名', '收货人电话', '收货省名称_y',
       '收货市名称_y', '区县名称', '收货人详细地址_y'],
      dtype='object')

In [189]:
# 去重订单可能会与日报数据存在些许出路，这是因为该用户在多个商家或渠道下过订单，去重的时候仅会保留一个订单
merchant_list = ['澄心优租', '优优2店', '北京海鸟窝科技有限公司', '租着用电脑数码', '趣智数码', '汇客好租(支付宝)', '汇客好租(抖音)', '乙辉数码','小蚂蚁租机', '兴鑫兴通讯', '呱子笔记本电脑', '南京聚格网络科技', '人人享租', '喜卓灵租机', '喜卓灵新租机', '星晟数码(支付宝)', '星晟数码(京东)', '蘑菇时间','云启德曜', '艾欧尼亚数码', '智云享租', '谦莲科技']
df_merge = df_contain.merge(df_merchant, left_on='订单id', right_on='order_id', how='left')
df_merge['商家'] = np.where((df_merge.商家=='星晟数码')&(df_merge.order_type=='ZFB_ORDER'), '星晟数码(支付宝)',
                        np.where((df_merge.商家=='星晟数码')&(df_merge.order_type=='JD_ORDER'), '星晟数码(京东)',
                            np.where((df_merge.商家=='汇客好租')&(df_merge.order_type=='ZFB_ORDER'), '汇客好租(支付宝)',
                                np.where((df_merge.商家=='汇客好租')&(df_merge.order_type=='DY_ORDER'), '汇客好租(抖音)', df_merge.商家))))               
# 删除商家为云启德曜中reject_check不为'1'的数据
# df_merge = df_merge[(df_merge.商家!='云启德曜')|((df_merge.商家=='云启德曜')&(df_merge.reject_check=='1')&(df_merge.下单日期>='2025-11-09'))|(df_merge.商家=='云启德曜')&(df_merge.下单日期<'2025-11-09')]
df_merge = df_merge[(df_merge.商家!='云启德曜')|((df_merge.商家=='云启德曜')&(df_merge.reject_check=='1'))]
df_merchant = df_merge[df_merge.商家.isin(merchant_list)]
# df_merchant = df_contain[df_contain.商家.isin(merchant_list)]
df_merchant.loc[:, '是否进件'] = np.where(df_merchant.进件=='进件', 1, 0)
df_merchant.loc[:, '出库'] = np.where(df_merchant.审核状态=='出库', 1, 0)
df_merchant_group = df_merchant.groupby(['月份', '商家']).agg({'单号': 'count', '是否进件': 'sum', '出库': 'sum', '总租金': 'mean', '买断金额': 'mean'}).rename(columns={'单号': '去重订单数', '是否进件': '进件数', '总租金': '租金平均价', '买断金额': '买断平均价'})
df_merchant_group.loc[:, '进件出库率'] = (df_merchant_group.出库/df_merchant_group.进件数).map(lambda x: format(x,".2%"))
df_merchant_group = df_merchant_group.reset_index()
df_merchant_group[['月份', '商家', '去重订单数', '进件数', '出库', '进件出库率', '租金平均价', '买断平均价']]

,月份,商家,去重订单数,进件数,出库,进件出库率,租金平均价,买断平均价
0,12,云启德曜,790,790,32,4.05%,6650.058532,13246.338861
1,12,北京海鸟窝科技有限公司,4,0,0,nan%,294.000000,1282.500000
2,12,星晟数码(京东),1387,1274,14,1.10%,4768.675761,9680.138861
3,12,星晟数码(支付宝),941,171,1,0.58%,4785.609819,8948.340680
4,12,智云享租,3907,3907,3,0.08%,8734.627630,14203.111646
5,12,汇客好租(支付宝),41,10,0,0.00%,4423.174146,9519.853659
6,12,租着用电脑数码,78,23,1,4.35%,1532.256410,3427.974359
7,12,艾欧尼亚数码,1,1,0,0.00%,115.000000,4200.000000
8,12,蘑菇时间,29,16,6,37.50%,983.793793,4459.589655
9,12,谦莲科技,366,58,4,6.90%,5049.856585,9131.133880


In [190]:
# 方案一：设置na参数，将NaN值视为False
# df_merge[df_merge.商家.str.contains('星晟数码', na=False)]
df_merge[(df_merge.商家=='星晟数码')].columns#.order_type.value_counts()

Index(['订单id', '单号', '外部订单号', '商家', '下单用户', '下单手机', '下单用户身份证号', '用户认证情况',
       '开始时间', '结束时间', '订单类型', '租期', '订单逾期天数', '总租金', '总押金', '商品押金', '总保险',
       '总运费', '总优惠', '订单状态', '是否买断', '下单时间', '收货人', '收货电话', '预授权支付单号', '活动名称',
       '收货省名称_x', '收货市名称_x', '收货区县名称', '收货人详细地址_x', '收货地址', '发货快递', '快递单号',
       '发货时间', '下单备注', '商品型号', '商品id', '下单商品', '商品属性', '商品数量', '租金单价', '买断金额',
       '预授权占用自有资金', '来源渠道', '备注信息', '用户实际支付金额', '联系人', '是否购买服务商品：1、否；2、是',
       '租物等级', '决策结果', '订单完成时间', '商品溢价费', '设置买断金额', '审核状态', '是否为资方订单',
       '第一期实付金额', '第二期应付金额', '第三期应付金额', '下单日期', '月份', '拒绝理由', '取消原因', '电审拒绝原因',
       '商品类型', '租赁方案', '押金类型', '是否使用优惠券', '是否进行预授权', '内存', '颜色', '进件', '归属渠道',
       '状态编码', '服务订单号', '租赁订单号', '服务商品名称', '服务商品价格', '服务商品改后价格', '服务订单状态',
       '服务时长（0表示长期）', '服务开始日期', '服务结束日期', '收货人姓名', '收货人电话', '收货省名称_y',
       '收货市名称_y', '区县名称', '收货人详细地址_y', 'order_id', 'order_type',
       'reject_check'],
      dtype='object')

In [191]:
# df_merchant_xcsm = df_contain[df_contain['商家']=='星晟数码']
# df_merchant_xcsm.loc[:, '是否进件'] = np.where(df_merchant_xcsm.进件=='进件', 1, 0)
# df_merchant_xcsm.loc[:, '出库'] = np.where(df_merchant_xcsm.审核状态=='出库', 1, 0)
# df_group = df_merchant_xcsm.groupby('下单日期').agg({'单号': 'count', '是否进件': 'sum', '出库': 'sum', '总租金': 'mean', '买断金额': 'mean'}).rename(columns={'单号': '去重订单数', '是否进件': '进件数', '总租金': '租金平均价', '买断金额': '买断平均价'})
# df_group

In [192]:
# df1 = pd.read_excel('F:/myfile/p站数据/当月数据/0801-0815.xlsx')
# # df0 =df1.copy()
# df1.shape

In [193]:
# df0_xcsm = df1[df1['商家']=='星晟数码']
# df0_xcsm["下单日期"]=df0_xcsm["下单时间"].dt.date
# df0_xcsm["下单日期"]=pd.to_datetime(df0_xcsm["下单日期"],errors="coerce")
# df0_xcsm["下单日期"]
# df0_xcsm["月份"]=df0_xcsm["下单日期"].dt.month
# #删除重复单号
# df0_xcsm.drop_duplicates(subset=["单号"],inplace=True)
# #删除身份证空值行
# df0_xcsm.dropna(subset=["下单用户身份证号"],axis=0,inplace=True)
# #去刷单订单
# df0_xcsm.drop(df0_xcsm[df0_xcsm['备注信息'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
# df0_xcsm.drop(df0_xcsm[df0_xcsm['活动名称']=="1000单秘密计划"].index,inplace=True)
# df0_xcsm.drop(df0_xcsm[df0_xcsm['活动名称']=="1000单秘密计划-无优惠"].index,inplace=True)
# df0_xcsm.drop(df0_xcsm[df0_xcsm['活动名称']=="1000单曙光计划"].index,inplace=True)
# df0_xcsm.drop(df0_xcsm[df0_xcsm['活动名称']=="线下门店3个月试行"].index,inplace=True)
# #删除重复单号
# df0.drop_duplicates(subset=["单号"],inplace=True)
# #删除订单状态空值行
# df0_xcsm.dropna(subset=["订单状态"],axis=0,inplace=True)
# df0_xcsm.drop(df0_xcsm[df0_xcsm['下单用户'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华","邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文","王家伟"])].index,inplace=True)
# df0_xcsm.drop_duplicates(subset=["下单用户","下单手机","下单用户身份证号","下单日期"],keep="last",inplace=True)
# df0_xcsm.单号.count()

In [194]:
# df_xcsm_list = df0_xcsm1.单号.to_list()
# len(df_xcsm_list)

In [195]:
# df0 =df1.copy()
# df0["下单日期"]=df0["下单时间"].dt.date
# df0["下单日期"]=pd.to_datetime(df0["下单日期"],errors="coerce")
# df0["下单日期"]
# df0["月份"]=df0["下单日期"].dt.month


In [196]:
# #删除重复单号
# df0.drop_duplicates(subset=["单号"],inplace=True)
# #删除身份证空值行
# df0.dropna(subset=["下单用户身份证号"],axis=0,inplace=True)
# #去刷单订单
# df0.drop(df0[df0['备注信息'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
# df0.drop(df0[df0['活动名称']=="1000单秘密计划"].index,inplace=True)
# df0.drop(df0[df0['活动名称']=="1000单秘密计划-无优惠"].index,inplace=True)
# df0.drop(df0[df0['活动名称']=="1000单曙光计划"].index,inplace=True)
# df0.drop(df0[df0['活动名称']=="线下门店3个月试行"].index,inplace=True)
# #删除订单状态空值行
# df0.dropna(subset=["订单状态"],axis=0,inplace=True)

In [197]:
# df0.drop_duplicates(subset=["下单用户","下单手机","下单用户身份证号","下单日期"],keep="last",inplace=True)


In [198]:
# df0.drop(df0[df0['下单用户'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华","邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文","王家伟"])].index,inplace=True)



# 机型均值

In [199]:
# # 商品押金  签约价   采购价   
dfck2["型号内存"]=dfck2["商品型号"]+"_"+dfck2["内存_x"]
dfck2.groupby(['型号内存']).agg({'商品押金':'mean','买断金额':'mean','成本':'mean'})

,商品押金,买断金额,成本
型号内存,,,
AirPods Pro 3_公开版,1899.000000,2468.300000,1630.0
Apple Watch Series 11_GPS版46mm,3199.000000,4158.100000,2849.0
Apple Watch Ultra 3_GPS+蜂窝网络49mm,6499.000000,8447.400000,6027.0
MacBook Air_13英寸M416GB+512GB,9499.000000,11094.800000,7459.0
MacBook Air_13英寸M4【10核+10核】16GB+512GB,9499.000000,11094.800000,7459.0
OPPO Find X9 Pro_16GB+512GB,5999.000000,7797.500000,5463.0
OPPO Find X9_16GB+512GB,5299.000000,6887.600000,4784.0
iPhone 17 Pro Max_256GB,10056.502246,13796.940629,9967.795148
iPhone 17 Pro Max_512GB,11993.152047,16548.574678,12043.561404


In [200]:
# df_zulin = dfck2.groupby(['型号内存']).agg({'订单号':'count', '商品押金':'mean','买断金额':'mean','成本':'mean'})
# df_zulin.to_excel(f'F:/宏生/{excel_date[-1]}宏生转化金额.xlsx')